# Rest Service Demonstration

## DB Sections Request

In [55]:
from dask import dataframe as dd
from datetime import datetime
import pandas as pd

df = dd.read_parquet('../cache/solarpanel.parquet')
start_time = pd.to_datetime(datetime.fromtimestamp(1647116453595 / 1000))
end_time = pd.to_datetime(datetime.fromtimestamp(1647202853595 / 1000))

mask = (
  (df.index >= start_time) &
  (df.index <= end_time)
)

mf = df.loc[mask, []].compute()
mf = mf.reset_index()
mf["gap"] = mf["timestamp"].diff().dt.seconds > 5
mf["gap"] = mf["gap"].cumsum()

mf

,timestamp,gap
0,2022-03-13 19:24:46,0
1,2022-03-13 19:24:47,0
2,2022-03-13 19:24:48,0
3,2022-03-13 19:24:49,0
4,2022-03-13 19:24:50,0
...,...,...
3433,2022-03-13 21:20:49,5
3434,2022-03-13 21:20:50,5
3435,2022-03-13 21:20:51,5
3436,2022-03-13 21:20:52,5


In [57]:
import numpy as np

gk = mf.groupby("gap").agg({'timestamp' : [np.min, np.max]})
gk

timestamp                    
                   amin                amax
gap                                        
0   2022-03-13 19:24:46 2022-03-13 19:26:09
1   2022-03-13 19:26:22 2022-03-13 19:26:42
2   2022-03-13 19:27:08 2022-03-13 20:10:52
3   2022-03-13 21:08:22 2022-03-13 21:10:27
4   2022-03-13 21:10:47 2022-03-13 21:20:13
5   2022-03-13 21:20:39 2022-03-13 21:20:53

In [58]:
gk.to_numpy().astype('datetime64[s]').astype('int')

array([[1647199486, 1647199569],
       [1647199582, 1647199602],
       [1647199628, 1647202252],
       [1647205702, 1647205827],
       [1647205847, 1647206413],
       [1647206439, 1647206453]])

## DB Records Request

In [46]:
from dask import dataframe as dd
from datetime import datetime
import pandas as pd

MAX_RECORDS = 150

df = dd.read_parquet('../cache/solarpanel.parquet')
start_time = pd.to_datetime(datetime.fromtimestamp(1647116453595 // 1000))
end_time = pd.to_datetime(datetime.fromtimestamp(1647202853595 // 1000))

mask = (
  (df.index >= start_time) &
  (df.index <= end_time)
)

units = ["voltage", "input_current"]

mf = df.loc[mask, units]

if len(mf.index) > MAX_RECORDS:
    resample_freq = len(mf) // MAX_RECORDS
    mf = mf.compute().resample(str(resample_freq) + "S").mean()

    mf = mf.dropna()
    
    mf["timestamp"] = mf.index
    mf["gap"] = mf["timestamp"].diff().dt.seconds > resample_freq
    mf["gap"] = mf["gap"].cumsum()
    
mf

34


,voltage,input_current,timestamp,gap
timestamp,,,,
2022-03-13 19:24:30,13.461183,71.316992,2022-03-13 19:24:30,0
2022-03-13 19:25:04,13.398550,79.180226,2022-03-13 19:25:04,0
2022-03-13 19:25:38,13.595178,58.592392,2022-03-13 19:25:38,0
2022-03-13 19:26:12,13.530712,81.752458,2022-03-13 19:26:12,0
2022-03-13 19:26:46,13.562923,72.991610,2022-03-13 19:26:46,0
...,...,...,...,...
2022-03-13 21:18:24,13.496438,79.365655,2022-03-13 21:18:24,1
2022-03-13 21:18:58,13.418445,68.362660,2022-03-13 21:18:58,1
2022-03-13 21:19:32,13.625350,81.352302,2022-03-13 21:19:32,1


In [48]:
gk = mf.groupby("gap")["timestamp"].count().cumsum()
split_rows = gk.tolist()
split_rows

[82, 105]

In [50]:
import numpy as np

mf = mf[units].reset_index()
mf["timestamp"] = mf["timestamp"].values.astype("datetime64[s]").astype("int")
mf[units] = mf[units].applymap('{:,.2f}'.format)

dfs = np.split(mf, split_rows, axis=0)
dfs

[     timestamp voltage input_current
 0   1647199470   13.46         71.32
 1   1647199504   13.40         79.18
 2   1647199538   13.60         58.59
 3   1647199572   13.53         81.75
 4   1647199606   13.56         72.99
 ..         ...     ...           ...
 77  1647202088   13.65         73.44
 78  1647202122   13.51         81.72
 79  1647202156   13.50         75.66
 80  1647202190   13.51         79.59
 81  1647202224   13.55         76.75
 
 [82 rows x 3 columns],
       timestamp voltage input_current
 82   1647205692   13.57         82.10
 83   1647205726   13.44         69.33
 84   1647205760   13.56         76.91
 85   1647205794   13.43         81.53
 86   1647205828   13.53         75.63
 87   1647205862   13.58         70.85
 88   1647205896   13.46         79.99
 89   1647205930   13.41         65.29
 90   1647205964   13.43         71.25
 91   1647205998   13.55         84.68
 92   1647206032   13.40         89.47
 93   1647206066   13.62         67.96
 94   16472

In [9]:
mfs = []

for i in range(len(dfs) - 1):
      mfs.append(dfs[i].to_numpy().tolist())
        
mfs

[[[1646912595, '12.90', '26.26'],
  [1646912596, '13.13', '20.78'],
  [1646912597, '13.02', '30.59'],
  [1646912598, '13.59', '84.10'],
  [1646912599, '13.57', '22.17'],
  [1646912600, '13.84', '132.36'],
  [1646912601, '13.65', '2.29'],
  [1646912602, '13.32', '58.81'],
  [1646912603, '13.54', '14.09'],
  [1646912604, '13.70', '94.59'],
  [1646912605, '13.64', '76.57'],
  [1646912606, '13.40', '108.32'],
  [1646912607, '12.87', '130.31'],
  [1646912608, '14.17', '44.19'],
  [1646912609, '13.21', '42.45'],
  [1646912610, '13.04', '54.04'],
  [1646912611, '12.82', '14.41'],
  [1646912612, '13.68', '24.38'],
  [1646912613, '13.30', '77.85'],
  [1646912614, '13.86', '40.85'],
  [1646912615, '12.92', '44.86']],
 [[1646912636, '13.57', '84.81'],
  [1646912637, '13.94', '115.31'],
  [1646912638, '14.25', '22.63'],
  [1646912639, '12.89', '146.27'],
  [1646912640, '12.65', '86.26'],
  [1646912641, '13.59', '126.34'],
  [1646912642, '13.56', '121.65'],
  [1646912643, '12.85', '56.35'],
  [1646